データ可視化を行うノートブック

# 岡本編集

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
train_df = pd.read_csv('./data/train.csv')
train_df.head()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
fig,ax = plt.subplots()
sns.countplot(x='score',data=train_df,ax=ax)
ax.set_title('スコアごとのデータ件数')
plt.show()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
train_df['text_len'] = train_df.full_text.str.len()
train_df.head()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
#文章が長いほどスコアが高いのではないか
fig,ax = plt.subplots()
ax.scatter(train_df.score,train_df.text_len,alpha=0.3)
ax.set_xlabel('スコア')
ax.set_ylabel('文字列の長さ')
ax.set_title('スコア VS 文字列の長さ')
plt.show()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
train_df['space_count'] = train_df.full_text.str.count(' ')
train_df['word_len_avg'] = (train_df.text_len - train_df.space_count) / (train_df.space_count + 1)
train_df.head()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
#一節が長めだとスコアが高いのではないか？
fig, ax = plt.subplots()
ax.scatter(train_df.score,train_df.word_len_avg,alpha=0.3)
ax.set_title('スコア VS 平均的な一節の文字列長さ')
ax.set_xlabel('スコア')
ax.set_ylabel('平均文字列長')
plt.show()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
#「I」で始まる小論文は自分が足りでスコアが低いのではないか
train_df['I-cnt'] = train_df.full_text.str.startswith('I') + train_df.full_text.str.count('. I ')
train_df.head()

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
fig,ax = plt.subplots()
ax.scatter(train_df.score,train_df['I-cnt'],alpha=0.3)
ax.set_title('スコア VS I-cnt')
ax.set_xlabel('スコア')
ax.set_ylabel('I-cnt')
plt.show()

In [ ]:
for feature in train_df.select_dtypes(include=int).columns:
    sns.histplot(x=feature,data=train_df,kde=True)
    plt.show()

In [ ]:
train_df['log_text_len'] = np.log(train_df.text_len)
train_df['log_space_count'] = np.log(train_df.space_count)

In [ ]:
sns.histplot(x='log_text_len', data=train_df,kde=True)
plt.show()

In [ ]:
sns.histplot(x='log_space_count',data=train_df,kde=True)
plt.show()

# 秋野編集

## ライブラリのインポート

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
import matplotlib.pyplot as plt
import japanize_matplotlib
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

## データ読み込み

In [ ]:
# データフォルダの場所を設定
dataPath = "/data"

# データの読み出し
df = pd.read_csv(dataPath + "/train.csv")
scores = df["score"]
texts = df["full_text"]

## 全体の単語の使用回数を見てみる

In [ ]:
# テキストをトークン化
tokenized_texts = [word_tokenize(text.lower()) for text in texts]

In [ ]:
# データ確認
# pd.DataFrame(tokenized_texts).head()

In [ ]:
# 単語の使用回数を辞書として抽出する
word_dic = {}
for text in tokenized_texts:
    for word in text:
        if word in word_dic:
            word_dic[word] += 1
        else:
            word_dic[word] = 1

In [ ]:
# 単語の使用回数をDataFrame型に変換する
# 回数の多い順でソートしておく
sorted_data = sorted(word_dic.items(), key=lambda item: item[1], reverse=True)
sorted_word_dic = dict(sorted_data)

# 辞書をDataFrameに変換
word_df = pd.DataFrame(list(sorted_word_dic.items()), columns=['word', 'count'])

In [ ]:
# データ確認
word_df.head()

In [ ]:
# 単語の使用回数をグラフ表示する
fig, ax = plt.subplots(figsize=(15,6))
ax.bar(x=word_df['word'][:30], height=word_df['count'][:30])
ax.set_xlabel('単語')
ax.set_ylabel('使用回数')
ax.set_title('全テキストの単語別使用回数 上位のみ')
plt.show()

## スコアによって、使用単語の傾向が変わるか

使用単語回数と、スコアの相関関係を見る

In [ ]:
# 必要なNLTKデータのダウンロード
nltk.download('punkt')
nltk.download('stopwords')

# stopword(意味のない単語)を設定
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 小文字に変換
    text = text.lower()
    # 単語にトークン化
    words = word_tokenize(text)
    # ストップワードの除去
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(words)

In [ ]:
# データの読み出し
df = pd.read_csv(dataPath + "/train.csv")
df = df.head(8000).copy() # データ少な目で実験。全データにするとなぜかクラッシュする
scores = df["score"]
texts = df["full_text"]

df['processed_text'] = df['full_text'].apply(preprocess_text)

In [ ]:
# CountVectorizerを使って単語の出現頻度をベクトル化
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_text'])

# ベクトルをデータフレームに変換
words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# スコアを追加
words_df['score'] = df['score']

# データ確認
# words_df

In [ ]:
# 相関計算のために単語列のみを抽出
words_only_df = words_df.drop(columns=['score'])

# 相関を計算し、結果をDataFrameに格納
correlations = {}
for word in words_only_df.columns:
    correlation, _ = pearsonr(words_only_df[word], words_df['score'])
    correlations[word] = correlation

# 相関(絶対値)の高い順にソート
sorted_correlations = sorted(correlations.items(), key=lambda item: item[1], reverse=True)

# 相関上位の単語とその相関値
top_words = pd.DataFrame(sorted_correlations, columns=['word', 'correlation'])

# データ確認
top_words.head()

In [ ]:
# 正の相関の強い単語をグラフ表示
fig, ax = plt.subplots(figsize=(20,6))
ax.bar(x=top_words['word'][:50], height=top_words['correlation'][:50])
ax.set_xlabel('単語')
ax.set_ylabel('相関係数')
ax.set_title('単語の出現回数とスコアの相関関係')
plt.xticks(rotation=90, fontsize=20)
plt.show()

In [ ]:
# 負の相関の強い単語をグラフ表示
fig, ax = plt.subplots(figsize=(20,6))
ax.bar(x=top_words['word'][-50:], height=top_words['correlation'][-50:])
ax.set_xlabel('単語')
ax.set_ylabel('相関係数')
ax.set_title('単語の出現回数とスコアの相関関係')
plt.xticks(rotation=90, fontsize=20)
plt.show()

正の相関にしろ、負の相関にしろ、大した相関関係はなさそう  
正の相関の上位は少し相関があるが、あまりキーワード的な言葉には見えない